# Оценка точности модели, переобучение, регуляризация

In [ ]:
Дана статистика пользователей adult.csv.
Получите значения AUC для различных моделей и их параметров.
Используйте как минимум 3 различные модели классификации.
Решением будет ссылка на гитхаб с ноутбуком.

In [1]:
import pandas as pd

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('adult.csv')

In [5]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [6]:
df.shape

(48842, 15)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [8]:
df.workclass.value_counts()

Private             33906
Self-emp-not-inc     3862
Local-gov            3136
?                    2799
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: workclass, dtype: int64

In [9]:
df['workclass'] = df['workclass'].str.replace('?', 'Other')

In [10]:
df['marital-status'].value_counts()

Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: marital-status, dtype: int64

In [11]:
df.race.value_counts()

White                 41762
Black                  4685
Asian-Pac-Islander     1519
Amer-Indian-Eskimo      470
Other                   406
Name: race, dtype: int64

In [12]:
df['native-country'].value_counts().head()

United-States    43832
Mexico             951
?                  857
Philippines        295
Germany            206
Name: native-country, dtype: int64

In [13]:
df['native-country'] = df['native-country'].str.replace('?', 'United-States')

In [14]:
df.gender = df.gender.replace({'Male': 1, 'Female': 0})

In [15]:
df.gender.value_counts()

1    32650
0    16192
Name: gender, dtype: int64

In [16]:
df.education.value_counts()

HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: education, dtype: int64

In [17]:
df.relationship.value_counts()

Husband           19716
Not-in-family     12583
Own-child          7581
Unmarried          5125
Wife               2331
Other-relative     1506
Name: relationship, dtype: int64

In [18]:
df.occupation.value_counts()

Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
?                    2809
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: occupation, dtype: int64

In [19]:
df['occupation'] = df.occupation.replace('?', 'Other')

In [20]:
df.income.value_counts()

<=50K    37155
>50K     11687
Name: income, dtype: int64

In [21]:
df['income'] = df.income.replace({'<=50K': 0, '>50K': 1})

In [22]:
df.income.value_counts()

0    37155
1    11687
Name: income, dtype: int64

In [23]:
df = df.drop(columns='education')

In [24]:
df.head()

,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,7,Never-married,Machine-op-inspct,Own-child,Black,1,0,0,40,United-States,0
1,38,Private,89814,9,Married-civ-spouse,Farming-fishing,Husband,White,1,0,0,50,United-States,0
2,28,Local-gov,336951,12,Married-civ-spouse,Protective-serv,Husband,White,1,0,0,40,United-States,1
3,44,Private,160323,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,1,7688,0,40,United-States,1
4,18,Other,103497,10,Never-married,Other,Own-child,White,0,0,0,30,United-States,0


In [25]:
df_type_object = df.select_dtypes(include='object')
df_type_object.columns

Index(['workclass', 'marital-status', 'occupation', 'relationship', 'race',
       'native-country'],
      dtype='object')

In [26]:
data_dummies = pd.get_dummies(df, columns = df_type_object.columns)
data_dummies.head()

,age,fnlwgt,educational-num,gender,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,1,0,0,50,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,1,0,0,40,1,0,1,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,1,7688,0,40,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,18,103497,10,0,0,0,30,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [27]:
y = data_dummies['income']

In [28]:
X = data_dummies.drop(columns='income')

In [29]:
from sklearn.preprocessing import StandardScaler

In [30]:
sc = StandardScaler()

In [31]:
X_sc = sc.fit_transform(X)

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.3, random_state=999)

In [34]:
X_train.shape, y_train.shape

((34189, 90), (34189,))

In [35]:
X_test.shape, y_test.shape

((14653, 90), (14653,))

In [36]:
from sklearn.tree import DecisionTreeClassifier

In [37]:
model1 = DecisionTreeClassifier()

In [38]:
params = [{'max_depth': list(range(1,20))}]

In [39]:
tree = GridSearchCV(model1, param_grid = params, scoring='roc_auc')

In [40]:
tree.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                        13, 14, 15, 16, 17, 18, 19]}],
             scoring='roc_auc')

In [41]:
y_pred = tree.best_estimator_.predict(X_test)

In [42]:
fpr, tpr, thres = roc_curve(y_test, y_pred)
auc(fpr, tpr)

0.7444813143117301

In [43]:
tree.cv_results_

{'mean_fit_time': array([0.04698157, 0.06260991, 0.08418064, 0.10703273, 0.12502923,
        0.14480548, 0.16966877, 0.17912679, 0.19991159, 0.2088892 ,
        0.22306581, 0.23798223, 0.24695053, 0.25827794, 0.26738315,
        0.27534075, 0.28143115, 0.28729973, 0.29139395]),
 'std_fit_time': array([0.00905383, 0.00052113, 0.00071695, 0.00170383, 0.0003283 ,
        0.00106921, 0.00977126, 0.00191   , 0.00966168, 0.00182771,
        0.00246956, 0.00385287, 0.00310931, 0.00428206, 0.00646179,
        0.00637221, 0.00722845, 0.00953128, 0.00972811]),
 'mean_score_time': array([0.00415978, 0.00386839, 0.00396099, 0.00410614, 0.00402288,
        0.00408845, 0.00418615, 0.00421724, 0.00433078, 0.00425496,
        0.00427103, 0.00431633, 0.00435286, 0.00437346, 0.00443811,
        0.00447035, 0.00445585, 0.00461559, 0.00451808]),
 'std_score_time': array([6.86824739e-04, 3.69025136e-05, 8.23480791e-05, 2.06325273e-04,
        5.41619428e-05, 4.28743697e-05, 9.03857380e-05, 4.56905056e-05,


In [44]:
(tree.cv_results_['split0_test_score'][7]+tree.cv_results_['split1_test_score'][7]+tree.cv_results_['split2_test_score'][7]+tree.cv_results_['split3_test_score'][7]+tree.cv_results_['split4_test_score'][7])/5

0.9039208647122265

In [45]:
print('AUC на тестовом наборе: {:.3f}'.format(roc_auc_score(y_test, y_pred)))
print('Наилучшие значения параметров: {}'.format(tree.best_params_))
print('Наилучшее значение кросс-валид.: {:.3f}'.format(tree.best_score_))
print('Наилучшая модель: {}'.format(tree.best_estimator_))

AUC на тестовом наборе: 0.744
Наилучшие значения параметров: {'max_depth': 8}
Наилучшее значение кросс-валид.: 0.904
Наилучшая модель: DecisionTreeClassifier(max_depth=8)


In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
model2 = LogisticRegression()

In [48]:
params2 = [{'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10]}]

In [49]:
log_reg = GridSearchCV(model2, param_grid = params2, scoring='roc_auc')

In [50]:
log_reg.fit(X_train, y_train)

GridSearchCV(estimator=LogisticRegression(),
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10],
                          'penalty': ['l1', 'l2']}],
             scoring='roc_auc')

In [51]:
y_pred = log_reg.best_estimator_.predict(X_test)

In [52]:
print('AUC на тестовом наборе: {:.3f}'.format(roc_auc_score(y_test, y_pred)))
print('Наилучшие значения параметров: {}'.format(log_reg.best_params_))
print('Наилучшее значение кросс-валид.: {:.3f}'.format(log_reg.best_score_))
print('Наилучшая модель: {}'.format(log_reg.best_estimator_))

AUC на тестовом наборе: 0.763
Наилучшие значения параметров: {'C': 0.1, 'penalty': 'l2'}
Наилучшее значение кросс-валид.: 0.908
Наилучшая модель: LogisticRegression(C=0.1)


In [53]:
from sklearn.ensemble import RandomForestClassifier

In [54]:
model3 = RandomForestClassifier()

In [55]:
params3 = [{'n_estimators': [10, 20, 50, 100], 'max_depth': [3, 5, 7, 9]}]

In [56]:
forest = GridSearchCV(model3, param_grid = params3, scoring='roc_auc')

In [57]:
forest.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [3, 5, 7, 9],
                          'n_estimators': [10, 20, 50, 100]}],
             scoring='roc_auc')

In [58]:
y_pred = forest.best_estimator_.predict(X_test)

In [59]:
print('AUC на тестовом наборе: {:.3f}'.format(roc_auc_score(y_test, y_pred)))
print('Наилучшие значения параметров: {}'.format(forest.best_params_))
print('Наилучшее значение кросс-валид.: {:.3f}'.format(forest.best_score_))
print('Наилучшая модель: {}'.format(forest.best_estimator_))

AUC на тестовом наборе: 0.742
Наилучшие значения параметров: {'max_depth': 9, 'n_estimators': 100}
Наилучшее значение кросс-валид.: 0.911
Наилучшая модель: RandomForestClassifier(max_depth=9)
